# Batch Normalization

- a.k.a. BatchNorm, BN
- [Paper](https://arxiv.org/abs/1502.03167): Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift
- Sergey Ioffe et al. (Google), 2015
- Refs
  - [1] https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c
  - [2] https://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad
  - [3] https://shuuki4.wordpress.com/2016/01/13/batch-normalization-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EA%B5%AC%ED%98%84/
  - [4] https://www.analyticsvidhya.com/blog/2017/07/covariate-shift-the-hidden-problem-of-real-world-data-science/

핵심은 매 layer마다 값을 정규분포로 바꿔서 내리는 것.
그러면,
- LR을 좀 높게 써도 된다. 10배에서 30배까지 늘려도 됐다
- dropout을 아주 약간만 적용해도 된다 (5-10 퍼센트 정도)

## from [4]
- Dataset shift
  - Training set과 test set에 차이가 있을 때 (두 집합이 다른 분산으로부터 추출됐을 때) 사실 그걸 만족하는 학습을 하는 건 불가능
- 어떻게 해야 할까?
  - dataset 정제를 위해 일부 sample을 drop하거나 특정 feature를 drop할 수 있을 것
  - 하지만 정확히 버리는 것도 쉽지 않을 것
- 논문 제목에 있는 covariate shift는 dataset shift의 한 종류라고 함
  - "shift in the independent variables" (?)

## from the paper & [2]
- Input에서의 작은 차이는 NN의 # of layers가 많을수록 파장이 클 것
- Internal covariate shift
  - 매 계층에서의 activation의 distribution이 끊임없기 변하는 것
- 매 layer마다 입력이 같은 distribution이 되도록 하면 어떨까? 학습이 안정적으로 되지 않을까?
  - 입력이 $N(0, \sigma^{2})$를 따르면 학습이 잘된다는 연구가 이미 있다고 (르쿤 형님의 98년 논문)
  - 전체 feature의 상관관계를 구하고 정규화할 수도 있겠지만 수학적으로 복잡하다고
  - 위에서 말한 르쿤 형님 논문을 보면 feature 간에 상관관계가 있더라도 개별적으로 정규화하면 학습 속도에 도움이 된다고 함
- 각 feature를 개별적으로 $N(0, 1)$을 따르게 변환
  - 변환을 위한 $\mu$와 $\sigma$는 전체 batch에서 가져오는 것이 아니라(역시 수학적으로 부담), mini-batch로부터 추출
- 이걸 그대로 사용하지는 않고 scale & shift 수행

### how to train
- 모든 값을 $N(0, 1)$로 정규화 해버리면 non-linearity 효과가 감소하기 때문이라고
- 왜? sigmoid 함수의 모양을 생각해보면, 값이 0 주변에 몰리면 직선처럼 되잖아[3]
  - 그럴싸 하긴 한데... 다른 activation을 사용하면?
- 여튼 $N(0, 1)$ 변환 뒤, 다시 $y = \gamma x + \beta$ 적용
  - $\gamma$와 $\beta$는 학습하도록 함 (논문에서, 두 값이 backprop 가능(미분 가능)함을 간단히 식으로 표현)

### how to infer
- Training 시 얻은 $\mu$와 $\sigma$의 moving average를 대표값으로 사용

### 추가 고려사항
- BN을 적용하면 $N(0,1)$로 변환하기 때문에 bias가 별로 의미가 없을 수 있다고
  - 일리가 있다: bias로 인한 shift가 우선 무시된 후 *scale & shift* 단계에서 $\beta$로 shift될 것
- CNN에서는 filter(channel) 마다 BN을 해주는 형태로 하면 된다고
- Layer Norm에서 이야기한 것처럼 BN을 RNN에 적용하는 건 좀 어려울 것 같긴 함
  - 논문 말미에서는 RNN에도 적용해보고 싶다고 이야기하긴 했다...

## from [1]
- 각 계층이 더 독립적인 특징을 표현할 수 있게 된다고 한다. 하지만 이유를 설명하지는 않았다.
- 그리고 reg 효과가 발생하는 것은, 정규화를 하면서 hidden layer에 noise가 끼기 때문이라고 하는데, 이것도 설명이 좀 애매한 듯
- Dropout을 완전히 걷어내기 보다는 살짝 섞는 것이 효과가 좋다고 말하고 있다

## from [2]
- 데이터를 정규화하는 이유를 생각해보면, sigmoid 등의 activation으로 값이 내부적으로 특정 범위로 줄어버리니 미리 $N(0, 1)$로 input data를 normalize하는 것이 도움이 될거라는 것
- 이걸 매 layer마다 해준다고 생각하면 되겠다

In [1]:
import tensorflow as tf

tf.enable_eager_execution()
keras = tf.keras

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [3]:
def get_model(with_bn:bool=False):
    inp = keras.layers.Input(shape=(28, 28,))
    x = keras.layers.Flatten()(inp)
    x = keras.layers.Dense(1024)(x)
    if with_bn:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(tf.nn.relu)(x)
    x = keras.layers.Dense(1024)(x)
    if with_bn:
        x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(tf.nn.relu)(x)
    x = keras.layers.Dense(10)(x)
    out = keras.layers.Activation(tf.nn.softmax)(x)

    return keras.models.Model(inputs=inp, outputs=out)

In [4]:
model = get_model(with_bn=False)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, shuffle=True, batch_size=128)

Instructions for updating:
Colocations handled automatically by placer.
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 51us/sample - loss: 0.4660 - acc: 0.8328 - val_loss: 0.3959 - val_acc: 0.8570
Epoch 2/20
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3499 - acc: 0.8714 - val_loss: 0.3770 - val_acc: 0.8629
Epoch 3/20
60000/60000 [==============================] - 2s 38us/sample - loss: 0.3114 - acc: 0.8838 - val_loss: 0.3722 - val_acc: 0.8656
Epoch 4/20
60000/60000 [==============================] - 2s 39us/sample - loss: 0.2876 - acc: 0.8930 - val_loss: 0.3409 - val_acc: 0.8796
Epoch 5/20
60000/60000 [==============================] - 2s 40us/sample - loss: 0.2720 - acc: 0.8983 - val_loss: 0.3544 - val_acc: 0.8751
Epoch 6/20
60000/60000 [==============================] - 2s 38us/sample - loss: 0.2543 - acc: 0.9037 - val_loss: 0.3412 - val_acc: 0.8767
Epoch 7/20
60000/60000 [====================

In [5]:
model = get_model(with_bn=True)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, shuffle=True, batch_size=128)

Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
60000/60000 [==============================] - 4s 59us/sample - loss: 0.4348 - acc: 0.8444 - val_loss: 0.4735 - val_acc: 0.8294
Epoch 2/20
60000/60000 [==============================] - 3s 50us/sample - loss: 0.3216 - acc: 0.8809 - val_loss: 0.4273 - val_acc: 0.8467
Epoch 3/20
60000/60000 [==============================] - 3s 50us/sample - loss: 0.2809 - acc: 0.8945 - val_loss: 0.4579 - val_acc: 0.8520
Epoch 4/20
60000/60000 [==============================] - 3s 50us/sample - loss: 0.2562 - acc: 0.9022 - val_loss: 0.4150 - val_acc: 0.8524
Epoch 5/20
60000/60000 [==============================] - 3s 51us/sample - loss: 0.2327 - acc: 0.9128 - val_loss: 0.3569 - val_acc: 0.8785
Epoch 6/20
60000/60000 [==============================] - 3s 52us/sample - loss: 0.2161 - acc: 0.9183 - val_loss: 0.3465 - val_acc: 0.8810
Epoch 7/20
60000/60000 [==============================] - 3s 50us/sa

In [6]:
bn1 = model.get_layer(index=3)
bn1.get_config()

{'axis': [1],
 'beta_constraint': None,
 'beta_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}},
 'beta_regularizer': None,
 'center': True,
 'dtype': 'float32',
 'epsilon': 0.001,
 'gamma_constraint': None,
 'gamma_initializer': {'class_name': 'Ones', 'config': {'dtype': 'float32'}},
 'gamma_regularizer': None,
 'momentum': 0.99,
 'moving_mean_initializer': {'class_name': 'Zeros',
  'config': {'dtype': 'float32'}},
 'moving_variance_initializer': {'class_name': 'Ones',
  'config': {'dtype': 'float32'}},
 'name': 'batch_normalization_v1',
 'scale': True,
 'trainable': True}

In [7]:
bn = keras.models.Model(inputs=model.input, outputs=bn1.output)
bn.predict(x_test)

array([[-0.44869298,  0.07892215, -0.26858023, ..., -1.3953226 ,
         0.5594736 , -0.307774  ],
       [-1.3418978 , -2.949014  , -1.0502021 , ..., -0.668557  ,
        -2.867313  , -0.58744174],
       [-3.4107683 ,  0.46822828, -0.2145659 , ...,  2.3570657 ,
         0.26854962,  1.2525423 ],
       ...,
       [-0.19224165, -1.2856836 , -1.9171991 , ...,  0.10457253,
         1.3688984 ,  0.31426135],
       [-2.1334212 , -0.8088093 ,  0.9502682 , ...,  0.31572986,
         0.12444431,  0.4506183 ],
       [-0.4595825 , -0.00451753, -0.6693098 , ..., -0.9146105 ,
         0.3749123 ,  1.0066435 ]], dtype=float32)

In [8]:
bn1.beta

<tf.Variable 'batch_normalization_v1/beta:0' shape=(1024,) dtype=float32, numpy=
array([-0.33852214, -0.29313052, -0.22454068, ..., -0.27685034,
       -0.3549209 , -0.3226965 ], dtype=float32)>

In [9]:
bn1.gamma

<tf.Variable 'batch_normalization_v1/gamma:0' shape=(1024,) dtype=float32, numpy=
array([0.9952824 , 0.9475387 , 0.9208805 , ..., 0.9526772 , 0.82764727,
       0.9900742 ], dtype=float32)>

In [10]:
import numpy as np

In [12]:
(_7 - _8) / _9

<tf.Tensor: id=175585, shape=(10000, 1024), dtype=float32, numpy=
array([[-0.11069304,  0.3926517 , -0.0478233 , ..., -1.1740307 ,
         1.1048118 ,  0.0150721 ],
       [-1.0081317 , -2.8029287 , -0.89659995, ..., -0.41116408,
        -3.035583  , -0.26739937],
       [-3.0868084 ,  0.8035121 ,  0.01083178, ...,  2.7647517 ,
         0.75330466,  1.591031  ],
       ...,
       [ 0.14697385, -1.0475067 , -1.838087  , ...,  0.4003695 ,
         2.0827947 ,  0.64334357],
       [-1.8034067 , -0.5442298 ,  1.2757452 , ...,  0.6220157 ,
         0.5791902 ,  0.78106755],
       [-0.12163419,  0.3045923 , -0.48298246, ..., -0.6694399 ,
         0.8818167 ,  1.342667  ]], dtype=float32)>

In [17]:
np.mean(np.mean(_12, axis=0))

-0.015364518

In [16]:
np.mean(np.std(_12, axis=0))

1.0062392